In [62]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from torch import nn,optim
from torch.utils.data import DataLoader
from torchvision import datasets,transforms

In [63]:
batchsize=100

training_data=datasets.FashionMNIST(root="../fashion_mnist",train=True,transform=transforms.ToTensor(),download=True)
test_data=datasets.FashionMNIST(root="../fashion_mnist",train=False,transform=transforms.ToTensor(),download=True)

train_dataloader=DataLoader(training_data,batch_size=batchsize)
test_dataloader=DataLoader(test_data,batch_size=batchsize)

In [ ]:
#hyperparameters lwk
sequence_len=28
input_len=28
hidden_size=128
num_layer=2
num_classes=10
num_epochs=10
learning_rate=0.01

In [65]:
class LSTM(nn.Module):
   def __init__(self,input_len,hidden_size,num_classes,num_layer):
      super(LSTM,self).__init__()
      self.hidden_size=hidden_size
      self.num_layer=num_layer
      self.lstm=nn.LSTM(input_len,hidden_size,num_layer,batch_first=True)
      self.output_layer=nn.Linear(hidden_size,num_classes)

   def forward(self,X):
      hidden_states=torch.zeros(self.num_layer,X.size(0),self.hidden_size)
      cell_states=torch.zeros(self.num_layer,X.size(0),self.hidden_size)

      out,_ = self.lstm(X,(hidden_states,cell_states))
      out=self.output_layer(out[:,-1,:])
      return out

In [66]:
model=LSTM(input_len,hidden_size,num_classes,num_layer)
print(model)

LSTM(
  (lstm): LSTM(28, 128, num_layers=2, batch_first=True)
  (output_layer): Linear(in_features=128, out_features=10, bias=True)
)


In [67]:
loss_func=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=learning_rate)

In [68]:
#training loop

def train(num_epochs,model,train_dataloader,loss_func,optimizer):
    for epoch in range(num_epochs):
        for batch,(images,labels) in enumerate(train_dataloader):
            images=images.reshape(-1,sequence_len,input_len)
            outputs=model(images)
            loss=loss_func(outputs,labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if(batch+1)%100==0:
                print(f"Epoch [{epoch+1}/{num_epochs}],Step [{batch+1}/{len(train_dataloader)}],Loss:{loss.item():.4f}")


In [69]:
train(num_epochs,model,train_dataloader,loss_func,optimizer)

Epoch [1/20],Step [100/600],Loss:0.9209
Epoch [1/20],Step [200/600],Loss:0.7461
Epoch [1/20],Step [300/600],Loss:0.4398
Epoch [1/20],Step [400/600],Loss:0.4459
Epoch [1/20],Step [500/600],Loss:0.6706
Epoch [1/20],Step [600/600],Loss:0.3560
Epoch [2/20],Step [100/600],Loss:0.4388
Epoch [2/20],Step [200/600],Loss:0.3015
Epoch [2/20],Step [300/600],Loss:0.3675
Epoch [2/20],Step [400/600],Loss:0.4645
Epoch [2/20],Step [500/600],Loss:0.4749
Epoch [2/20],Step [600/600],Loss:0.2355
Epoch [3/20],Step [100/600],Loss:0.2927
Epoch [3/20],Step [200/600],Loss:0.3062
Epoch [3/20],Step [300/600],Loss:0.2655
Epoch [3/20],Step [400/600],Loss:0.3392
Epoch [3/20],Step [500/600],Loss:0.4389
Epoch [3/20],Step [600/600],Loss:0.2216
Epoch [4/20],Step [100/600],Loss:0.3039
Epoch [4/20],Step [200/600],Loss:0.2744
Epoch [4/20],Step [300/600],Loss:0.2544
Epoch [4/20],Step [400/600],Loss:0.2783
Epoch [4/20],Step [500/600],Loss:0.3929
Epoch [4/20],Step [600/600],Loss:0.1937
Epoch [5/20],Step [100/600],Loss:0.2747


In [70]:
#testing loop
with torch.no_grad():
    correct=0
    total=0
    acc=0
    best_acc=0
    for images,labels in test_dataloader:
        images=images.reshape(-1,sequence_len,input_len)
        outputs=model(images)
        _,predicted=torch.max(outputs,1)
        total+=labels.size(0)
        correct+=(predicted==labels).sum().item()
        acc=100*correct/total
        if acc>best_acc:
            best_acc=acc
            torch.save(model.state_dict(), 'best_lstm.pth')
    print(f"Accuracy of the model on the test images:{best_acc}%")

Accuracy of the model on the test images:76.77777777777777%
